In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import Adam
from torchvision import datasets, transforms
from SupConLosszy import SupConLoss
import copy


In [2]:
print('==> Preparing............................')
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
from torchvision.datasets import ImageFolder


import os
import torch
from torch import nn,optim
import torch.nn.functional as F
from torchvision import datasets, transforms

from time import perf_counter

import  numpy as np
import torch.utils.data as Data

from torch.utils.data import Dataset, DataLoader

class Safeman(Dataset):
    
    def __init__(self, data,targets):
        super(Safeman, self).__init__()
        self.data = data
        self.targets = targets
        
     
    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        img, target = self.data[idx], self.targets[idx]
        return img, target

class Safeman_Filter(Safeman):

    def __Filter__(self, known):
        targets = self.targets.data.numpy()
        mask, new_targets = [], []
        for i in range(len(targets)):
            if targets[i] in known:
                mask.append(i)
                new_targets.append(known.index(targets[i]))
        self.targets = np.array(new_targets)
        mask = torch.tensor(mask).long()
        self.data = torch.index_select(self.data, 0, mask)
        
class Safeman_FilterB(Safeman):

    def __Filter__(self, known):
        targets = self.targets.data.numpy()
        new_targets = []
        for i in range(len(targets)):
            if targets[i] in known:
                new_targets.append(0)
            else:
                new_targets.append(1)
        self.targets = np.array(new_targets)
        self.data = self.data

class Safeman_FilterC(Safeman):
    
    def __Filter__(self, trainknown):
        train_class_num=len(trainknown)
        for i in range(0,len(self.targets)) :
            if self.targets[i]>train_class_num:
                self.targets[i] = train_class_num
        self.data = self.data

        
def setup_seed(seed):

    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True

setup_seed(8)



known=[0, 1, 2,3,4]
unknown=[ 5,6,7]

num_class=len(known)



X_train0 = np.load('./UKM/X_train_ukm.npy')
y_train1 = np.load('./UKM/y_train_ukm.npy')
X_final_test0 = np.load('./UKM/X_final_test_ukm.npy' )
y__final_test1 = np.load('./UKM/y__final_test_ukm.npy')

X_train1=[]
X_final_test1=[]

for i in range(len(y_train1)):
    a = np.resize(X_train0[i], (1, 28, 28))
    X_train1 += [a]
    
for j in range(len(y__final_test1)):
    b = np.resize(X_final_test0[j], (1, 28, 28))
    X_final_test1 += [b]

i=0
j=0



x_train, x_test, y_train,y_test = torch.Tensor(X_train1), torch.Tensor(X_final_test1), torch.from_numpy(y_train1), torch.from_numpy(y__final_test1)

print(x_train.shape, x_test.shape, y_train.shape,y_test.shape)

train_dataset = Data.TensorDataset(x_train, y_train)
train_dataset.data = train_dataset.tensors[0]
train_dataset.targets = train_dataset.tensors[1]



test_dataset = Data.TensorDataset(x_test, y_test)
test_dataset.data = test_dataset.tensors[0]
test_dataset.targets = test_dataset.tensors[1]


labels =['ARP poisining', 'BeEF HTTP exploits','Mass HTTP requests','Metasploit exploits','Normal','Port scanning','TCP flood','UDP data flood']



train_dataset.classes = labels
test_dataset.classes = labels

train_dataset.classes_to_idx = {i: label for i, label in enumerate(labels)}
test_dataset.classes_to_idx = {i: label for i, label in enumerate(labels)}



b_s=16

trainset = Safeman_Filter(data=train_dataset.data,targets=train_dataset.targets)
print('All down Train Data:', len(trainset))
trainset.__Filter__(known=known)


train_loader = torch.utils.data.DataLoader(
    trainset, batch_size=b_s, shuffle=True,
    num_workers=4,drop_last=True)


print('Real train Data:', len(trainset))



testsetA = Safeman_Filter(data=test_dataset.data,targets=test_dataset.targets)
print('All testsetA Data:', len(testsetA))
testsetA.__Filter__(known=known)


test_loader_A = torch.utils.data.DataLoader(
    testsetA, batch_size=b_s, shuffle=True,
    num_workers=4,drop_last=True)


print('Real testsetA Data:', len(testsetA))


print("done!")

==> Preparing............................
torch.Size([10309, 1, 28, 28]) torch.Size([2578, 1, 28, 28]) torch.Size([10309]) torch.Size([2578])
All down Train Data: 10309
Real train Data: 8923
All testsetA Data: 2578
Real testsetA Data: 2226
done!


In [3]:
from torch.nn import Module
from torch import nn
import numpy as np
import torch
from torchvision.datasets import mnist
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

class Modelnsl8(nn.Module):
    def __init__(self):

        super().__init__()
        self.fc1 = nn.Linear(28*28, 64) 
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, num_class) 
      
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x4=x
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x3=x
        feat = F.normalize(x3,dim=1)
        x = self.fc4(x)
        return feat,F.log_softmax(x, dim = 1)

In [4]:
train_loader2 = train_loader
test_loader2 = test_loader_A
model = Modelnsl8()
sgd = SGD(model.parameters(), lr=1e-2)
criterion = SupConLoss(temperature=0.7)
loss_fn = CrossEntropyLoss()
all_epoch = 29
batch_size = b_s
for current_epoch in range(all_epoch):
    model.train()
    for idx, (train_x, train_label) in enumerate(train_loader2):
        sgd.zero_grad()
        features1,predict_y = model(train_x.float().view(-1, 28*28))
        
        features=torch.cat([features1, features1], dim=0)
        f1, f2 = torch.split(features, [batch_size, batch_size], dim=0)
        feat = torch.cat([f1.unsqueeze(1), f2.unsqueeze(1)], dim=1)         
        target2=train_label
        loss2 = criterion(feat, target2)
        loss1 = loss_fn(predict_y, train_label.long())
        loss=loss1+loss2
        if current_epoch > 20 :
            sgd = SGD(model.parameters(), lr=1e-3)
        if idx % 1000 == 0:
            print('idx: {}, loss: {}'.format(idx, loss.sum().item()))
        loss.backward()
        sgd.step()
    print("epoch i=",current_epoch+1)
    print("this train epoch end")
    all_correct_num = 0
    all_sample_num = 0
    model.eval()
    for idx, (test_x, test_label) in enumerate(test_loader2):
        _,predict_y = model(test_x.float().view(-1, 28*28))
        predict_y=predict_y.detach()
        predict_y = np.argmax(predict_y, axis=-1)
        current_correct_num = predict_y == test_label
        all_correct_num += np.sum(current_correct_num.numpy(), axis=-1)
        all_sample_num += current_correct_num.shape[0]
    acc = all_correct_num / all_sample_num
    print('accuracy: {:.3f}'.format(acc))
print("training end")


idx: 0, loss: 35.921024322509766
epoch i= 1
this train epoch end
accuracy: 0.841
idx: 0, loss: 33.20359420776367
epoch i= 2
this train epoch end
accuracy: 0.927
idx: 0, loss: 32.46283721923828
epoch i= 3
this train epoch end
accuracy: 0.946
idx: 0, loss: 33.24770736694336
epoch i= 4
this train epoch end
accuracy: 0.988
idx: 0, loss: 33.06803894042969
epoch i= 5
this train epoch end
accuracy: 0.991
idx: 0, loss: 28.88053321838379
epoch i= 6
this train epoch end
accuracy: 0.992
idx: 0, loss: 31.79766273498535
epoch i= 7
this train epoch end
accuracy: 0.990
idx: 0, loss: 29.600116729736328
epoch i= 8
this train epoch end
accuracy: 0.991
idx: 0, loss: 31.985748291015625
epoch i= 9
this train epoch end
accuracy: 0.977
idx: 0, loss: 33.674434661865234
epoch i= 10
this train epoch end
accuracy: 0.992
idx: 0, loss: 29.970869064331055
epoch i= 11
this train epoch end
accuracy: 0.994
idx: 0, loss: 31.778223037719727
epoch i= 12
this train epoch end
accuracy: 0.991
idx: 0, loss: 31.77687835693359